In [ ]:
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python
!pip install llama-index
!pip install transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.18-cp310-cp310-manylinux_2_35_x86_64.whl size=7069374 sha256=89cd26110ce3197d1f0dac29d67f650f1fa34f290abd816205e0f5f82acf808f
  Stored in directory: /root/.cache/pip/wheels/75/65/c3/54e9fca551b4734cc17f41fafb596a7807312fb470c806e1ab
Successfully built llama-cpp-python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.2/874.2 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.1 MB/s eta 0:00:

In [ ]:
!git clone https://github.com/PratyushChauhan/law_predictions.git

fatal: destination path 'law_predictions' already exists and is not an empty directory.


In [ ]:
!git clone https://github.com/PratyushChauhan/law_dataset.git


Cloning into 'law_dataset'...
remote: Enumerating objects: 16539, done.
remote: Total 16539 (delta 0), reused 0 (delta 0), pack-reused 16539
Receiving objects: 100% (16539/16539), 86.09 MiB | 15.94 MiB/s, done.
Resolving deltas: 100% (995/995), done.
Updating files: 100% (16776/16776), done.


In [ ]:
# !rm ./law_dataset/IN-Abs/test-data/summary/7*.txt
!ls ./law_dataset/IN-Abs/test-data/summary/

1181.txt  1789.txt  2304.txt  2727.txt	3442.txt  415.txt   4938.txt  5538.txt	6245.txt  6728.txt
1195.txt  1974.txt  232.txt   2796.txt	3531.txt  4316.txt  4963.txt  5597.txt	6270.txt  6778.txt
1329.txt  2035.txt  2392.txt  2913.txt	3542.txt  4451.txt  496.txt   5707.txt	6276.txt  6852.txt
1378.txt  2052.txt  2440.txt  3019.txt	3602.txt  4480.txt  5141.txt  5861.txt	6413.txt  6881.txt
1406.txt  2065.txt  2593.txt  314.txt	362.txt   4568.txt  5142.txt  5888.txt	6521.txt  690.txt
1522.txt  2122.txt  2609.txt  3168.txt	380.txt   4641.txt  5248.txt  5937.txt	652.txt   7109.txt
1531.txt  2124.txt  2627.txt  3210.txt	3844.txt  4782.txt  5266.txt  5994.txt	660.txt   7130.txt
1697.txt  2207.txt  2649.txt  3292.txt	3893.txt  4807.txt  5364.txt  6003.txt	6622.txt  715.txt
1762.txt  2248.txt  2657.txt  3356.txt	3924.txt  4860.txt  5397.txt  6118.txt	6647.txt  784.txt
1778.txt  2256.txt  266.txt   3436.txt	4071.txt  4917.txt  5471.txt  6157.txt	6668.txt  78.txt


<h1>initialize llm zephyr</h1>

In [ ]:
from pprint import pprint

In [ ]:
# import os

# # Directory path where your .txt files are located
# directory_path = './law_dataset/IN-Abs/test-data/summary/'

# # List all .txt files in the directory
# txt_files = [f for f in os.listdir(directory_path) if f.endswith('.txt')]

# # Sort the list of files to maintain order


# # Create a combined.txt file to write the flattened data
# with open(os.path.join(directory_path, 'combined.txt'), 'w') as combined_file:
#     # Iterate through each .txt file
#     for txt_file in txt_files:
#         with open(os.path.join(directory_path, txt_file), 'r') as file:
#             # Read and flatten the content of the file into one line
#             flattened_data = file.read().replace('\n', ' ')
#             # Write the flattened data to the combined file
#             combined_file.write(flattened_data + '\n')

# print("Flattened data from all files has been written to combined.txt")


In [ ]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.prompts import PromptTemplate
from llama_index.llms import HuggingFaceLLM

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


def messages_to_prompt(messages):
  prompt = ""
  for message in messages:
    if message.role == 'system':
      prompt += f"<|system|>\n{message.content}</s>\n"
    elif message.role == 'user':
      prompt += f"<|user|>\n{message.content}</s>\n"
    elif message.role == 'assistant':
      prompt += f"<|assistant|>\n{message.content}</s>\n"

  # ensure we start with a system prompt, insert blank if needed
  if not prompt.startswith("<|system|>\n"):
    prompt = "<|system|>\n</s>\n" + prompt

  # add final assistant prompt
  prompt = prompt + "<|assistant|>\n"

  return prompt


llm = HuggingFaceLLM(
    model_name="HuggingFaceH4/zephyr-7b-beta",
    tokenizer_name="HuggingFaceH4/zephyr-7b-beta",
    query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
    context_window=5000,
    max_new_tokens=1000,
    model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

<h1>Parse Documents</h1>

In [ ]:
from llama_index import ServiceContext

dir = './law_dataset/IN-Abs/test-data/judgement'

service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5",chunk_size=1024)



In [ ]:
from llama_index.response.notebook_utils import display_response

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.8 MB/s eta 0:00:00


In [ ]:
import re
from collections import Counter
from bert_score import score



# Initialize scores
total_bert_score = 0
total_precision_score = 0
total_recall_score = 0
total_f1_score = 0

# Lists to store precision and recall for BERTScore
precision_list = []
recall_list = []



In [ ]:
!mkdir pred

mkdir: cannot create directory ‘pred’: File exists


In [ ]:
import os
# directory_path = './law_predictions'
validation_path = './law_dataset/IN-Abs/test-data/summary/'
file_list = os.listdir(validation_path)

# for file_name in file_list:
#     file_path = os.path.join(directory_path, file_name)
#     validation_file = validation_path+file_name
#     # Check if the item is a file (not a subdirectory)
#     # with open(file_path, "r") as file:
#         # text = file.read()

#     with open(validation_file, "r") as file:
#         validation_text = file.read()
print(file_list)

['6668.txt', '415.txt', '7109.txt', '6157.txt', '1789.txt', '3019.txt', '496.txt', '1195.txt', '2657.txt', '5888.txt', '2207.txt', '1522.txt', '6118.txt', '2392.txt', '3210.txt', '2052.txt', '5248.txt', '380.txt', '5861.txt', '5141.txt', '6647.txt', '6852.txt', '1778.txt', '2035.txt', '1181.txt', '2627.txt', '2796.txt', '2065.txt', '3168.txt', '4641.txt', '78.txt', '2727.txt', '3844.txt', '6521.txt', '4860.txt', '266.txt', '6276.txt', '3542.txt', '5142.txt', '1762.txt', '6245.txt', '1329.txt', '3531.txt', '4480.txt', '1406.txt', '5538.txt', '362.txt', '784.txt', '2913.txt', '5597.txt', '660.txt', '4071.txt', '5364.txt', '232.txt', '2124.txt', '2649.txt', '6270.txt', '3924.txt', '690.txt', '7130.txt', '5266.txt', '652.txt', '3602.txt', '4316.txt', '5937.txt', '2609.txt', '5471.txt', '2304.txt', '6622.txt', '5707.txt', '3292.txt', '1378.txt', '2256.txt', '4807.txt', '3893.txt', '6413.txt', '3356.txt', '3442.txt', '6728.txt', '2248.txt', '1697.txt', '3436.txt', '6003.txt', '314.txt', '599

In [ ]:
!pip install rouge

In [ ]:
# import os
# from rouge import Rouge
# from bert_score import score

# # Define the directory paths
# directory_path = './pred'
# validation_path = './law_dataset/IN-Abs/test-data/summary/'
# file_list = os.listdir(directory_path)

# # Initialize lists to store scores
# total_precision_score = 0.0
# total_recall_score = 0.0
# total_f1_score = 0.0
# total_bert_score = 0.0
# precision_list = []
# recall_list = []
# bert_score_list = []

# # Initialize the ROUGE scorer
# rouge = Rouge()

# for file_name in file_list:
#     file_path = os.path.join(directory_path, file_name)
#     validation_file = os.path.join(validation_path, file_name)

#     # Check if the item is a file (not a subdirectory)
#     if os.path.isfile(file_path):
#         print("File:", file_name)
#         # Read text from the generated summary file
#         with open(file_path, "r") as file:
#             candidate = file.read()

#         # Read the reference summary text
#         with open(validation_file, "r") as file:
#             reference = file.read()

#         # Calculate ROUGE scores
#         rouge_scores = rouge.get_scores(candidate, reference)

#         # Extract precision, recall, and F1 scores
#         precision = rouge_scores[0]["rouge-1"]["r"]
#         recall = rouge_scores[0]["rouge-1"]["p"]
#         f1_score = rouge_scores[0]["rouge-1"]["f"]

#         # Update total ROUGE scores
#         total_precision_score += float(precision)
#         total_recall_score += float(recall)
#         total_f1_score += float(f1_score)

#         # Append precision and recall to the lists
#         precision_list.append(float(precision))
#         recall_list.append(float(recall))

#         # Calculate BERTScore
#         P, R, F1 = score([candidate], [reference], lang="en")
#         bert_score = F1.item()

#         # Update total BERTScore
#         total_bert_score += bert_score

#         # Append BERTScore to the list
#         bert_score_list.append(bert_score)

#         # Print scores neatly
#         print("ROUGE-1 Precision:", precision)
#         print("ROUGE-1 Recall:", recall)
#         print("ROUGE-1 F1 Score:", f1_score)
#         print("BERTScore:", bert_score)
#         print("----------------------")



# # Calculate average ROUGE scores
# num_files = len(file_list)
# avg_precision_score = total_precision_score / num_files
# avg_recall_score = total_recall_score / num_files
# avg_f1_score = total_f1_score / num_files

# # Calculate average BERTScore
# avg_bert_score = total_bert_score / num_files

# print(f'Average ROUGE Precision Score: {avg_precision_score:.6f}')
# print(f'Average ROUGE Recall Score: {avg_recall_score:.6f}')
# print(f'Average ROUGE F1 Score: {avg_f1_score:.6f}')
# print(f'Average BERTScore: {avg_bert_score:.6f}')

# # Print precision and recall for ROUGE and BERTScore
# for i, (precision, recall, bert_score) in enumerate(zip(precision_list, recall_list, bert_score_list)):
#     print(f'File {i + 1} - ROUGE Precision: {precision:.6f}, ROUGE Recall: {recall:.6f}, BERTScore: {bert_score:.6f}')


In [ ]:
# !pip install rouge

In [ ]:
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.agent import OpenAIAgent, ReActAgent
from llama_index import SummaryIndex
from llama_index import VectorStoreIndex
from llama_index import SimpleDirectoryReader
import os
from rouge import Rouge
from bert_score import score

directory_path = './law_predictions'
validation_path = './law_dataset/IN-Abs/test-data/summary/'
saveDir = './pred/'
# Initialize lists to store scores
total_precision_score = 0.0
total_recall_score = 0.0
total_f1_score = 0.0
total_bert_score = 0.0
precision_list = []
recall_list = []
bert_score_list = []

# Initialize the ROUGE scorer
rouge = Rouge()

for file_name in file_list:
    validation_file = os.path.join(validation_path, file_name)

    # Check if the item is a file (not a subdirectory)
    print("File:", file_name)
    # Read text from the judgement file
    # docs = '/content/law_dataset/IN-Abs/test-data/judgement/' + file_name
    file_path = '/content/pred/'+file_name

    if os.path.exists(file_path):
      print('skipping --------------')
      continue
    print("File exists!")
    docs = SimpleDirectoryReader(
        input_files=['/content/law_dataset/IN-Abs/test-data/judgement/' + file_name]
    ).load_data()
    summary_index = SummaryIndex.from_documents(docs, service_context=service_context)

    qe = summary_index.as_query_engine(response_mode="tree_summarize")
    response = qe.query('Summarize the current document. Do not miss or invent any details.')
    # print(response)
    candidate = str(response)
    with open('./pred/'+file_name, 'w') as pred_file:
            pred_file.write(candidate)

    # Read the reference summary text
    with open(validation_file, "r") as file:
        reference = file.read()

    # Calculate ROUGE scores
    rouge_scores = rouge.get_scores(candidate, reference)

    # Extract precision, recall, and F1 scores
    precision = rouge_scores[0]["rouge-1"]["r"]
    recall = rouge_scores[0]["rouge-1"]["p"]
    f1_score = rouge_scores[0]["rouge-1"]["f"]

    # Update total ROUGE scores
    total_precision_score += float(precision)
    total_recall_score += float(recall)
    total_f1_score += float(f1_score)

    # Append precision and recall to the lists
    precision_list.append(float(precision))
    recall_list.append(float(recall))

    # Calculate BERTScore
    P, R, F1 = score([candidate], [reference], lang="en")
    bert_score = F1.item()

    # Update total BERTScore
    total_bert_score += bert_score

    # Append BERTScore to the list
    bert_score_list.append(bert_score)

    # Print scores neatly
    print("ROUGE-1 Precision:", precision)
    print("ROUGE-1 Recall:", recall)
    print("ROUGE-1 F1 Score:", f1_score)
    print("BERTScore:", bert_score)
    print("----------------------")


# Calculate average scores
avg_bert_score = total_bert_score / 2
avg_precision_score = total_precision_score / 2
avg_recall_score = total_recall_score / 2
avg_f1_score = total_f1_score / 2

print(f'Average BERTScore: {avg_bert_score:.6f}')
print(f'Average ROUGE-1 Precision Score: {avg_precision_score:.6f}')
print(f'Average ROUGE-1 Recall Score: {avg_recall_score:.6f}')
print(f'Average ROUGE-1 F1 Score: {avg_f1_score:.6f}')


File: 6668.txt
skipping --------------
File: 415.txt
skipping --------------
File: 7109.txt
skipping --------------
File: 6157.txt
skipping --------------
File: 1789.txt
skipping --------------
File: 3019.txt
skipping --------------
File: 496.txt
skipping --------------
File: 1195.txt
skipping --------------
File: 2657.txt
skipping --------------
File: 5888.txt
skipping --------------
File: 2207.txt
skipping --------------
File: 1522.txt
skipping --------------
File: 6118.txt
skipping --------------
File: 2392.txt
skipping --------------
File: 3210.txt
skipping --------------
File: 2052.txt
skipping --------------
File: 5248.txt
skipping --------------
File: 380.txt
skipping --------------
File: 5861.txt
skipping --------------
File: 5141.txt
skipping --------------
File: 6647.txt
skipping --------------
File: 6852.txt
skipping --------------
File: 1778.txt
skipping --------------
File: 2035.txt
skipping --------------
File: 1181.txt
skipping --------------
File: 2627.txt
skipping ----

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should se

(…)o/roberta-large/resolve/main/config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

(…)co/roberta-large/resolve/main/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

(…)co/roberta-large/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

ROUGE-1 Precision: 0.17493472584856398
ROUGE-1 Recall: 0.536
ROUGE-1 F1 Score: 0.26377952384873526
BERTScore: 0.8384582996368408
----------------------
File: 4963.txt
File exists!


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE-1 Precision: 0.2125984251968504
ROUGE-1 Recall: 0.6
ROUGE-1 F1 Score: 0.3139534845085182
BERTScore: 0.833879828453064
----------------------
File: 4782.txt
File exists!


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE-1 Precision: 0.14153846153846153
ROUGE-1 Recall: 0.5168539325842697
ROUGE-1 F1 Score: 0.22222221884699764
BERTScore: 0.8304246664047241
----------------------
Average BERTScore: 16.796415
Average ROUGE-1 Precision Score: 4.256207
Average ROUGE-1 Recall Score: 10.596871
Average ROUGE-1 F1 Score: 5.768503
File 1 - BERTScore Precision: 0.053008, BERTScore Recall: 0.613793
File 2 - BERTScore Precision: 0.172414, BERTScore Recall: 0.290698
File 3 - BERTScore Precision: 0.180556, BERTScore Recall: 0.364486
File 4 - BERTScore Precision: 0.122150, BERTScore Recall: 0.688073
File 5 - BERTScore Precision: 0.513274, BERTScore Recall: 0.527273
File 6 - BERTScore Precision: 0.319277, BERTScore Recall: 0.430894
File 7 - BERTScore Precision: 0.183673, BERTScore Recall: 0.500000
File 8 - BERTScore Precision: 0.167683, BERTScore Recall: 0.491071
File 9 - BERTScore Precision: 0.208443, BERTScore Recall: 0.541096
File 10 - BERTScore Precision: 0.117647, BERTScore Recall: 0.537634
File 11 - BERTScor

In [ ]:
import tarfile

with tarfile.open('./pred.tar', 'w:gz') as tar:
    tar.add('./pred', arcname='pred')


In [ ]:
import tarfile

# Replace 'file_path/pred.tar' with the actual path to your 'pred.tar' file
file_path = '/content/pred.tar'

# Specify the directory where you want to extract the contents
extract_dir = '/content/pred'

# Open the tar file
with tarfile.open(file_path, 'r:gz') as tar:
    # Extract all contents to the specified directory
    tar.extractall(path=extract_dir)

print(f"Contents extracted to {extract_dir}.")


Contents extracted to /content/pred.
